In [1]:
# CREATE TABLE items (id bigserial PRIMARY KEY, content TEXT, embedding vector(1536));

In [2]:
import os
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
import openai
import psycopg2

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Set up your OpenAI API key
# openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
print(OPENAI_API_KEY[:12])

sk-proj-4_jr


In [5]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="host.docker.internal",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [6]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(input=prompt, model="text-embedding-ada-002")

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return "[" + ",".join(map(str, embedding)) + "]"

Second, find the most relevant movies for a provided user prompt by calculating the cosine distance (`<=>`) between the prompt's and movies' embeddings:


In [7]:
try:

    query = "What is out of the box technology"

    query_embedding = get_embedding(query)

    # Perform a cosine similarity search
    sql = """
        SELECT element_id, element_text, 1 - (embedding <=> %s) AS cosine_similarity
        FROM tbl_doc_elements
        WHERE element_type = 'NARRATIVETEXT'
        ORDER BY cosine_similarity DESC LIMIT 3;
        """
    cur.execute(sql, (query_embedding,))

    print("Query:", query)

    print("Most similar sentences:")

    for row in cur.fetchall():

        print(f"ID: {row[0]},  Cos: {row[2]}, CONTENT: {row[1]},")


except Exception as e:

    print("Error executing query", str(e))

Query: What is out of the box technology
Most similar sentences:
ID: cbb00b10741723c0e1aa057766000932,  Cos: -0.008235746402816435, CONTENT: But one size doesn’t fit all for search. You can utilize out-of-the-box technology, build your own with feature-rich, custom design and functionality, or anything in-between.,
ID: 1e10486a3f5515298c1e0cdbdebe41db,  Cos: -0.01675345808164619, CONTENT: Discover the blueprint for planning, designing, and building a search experience that meets your users’ needs, your team’s resources, and (of course) your budget. With an overview of the latest tech and real-world case studies that highlight what’s possible, you can envision your future search experience and bring it to life.,
ID: c2d7a7bc292ea16bd674aa1039a2aa15,  Cos: -0.06120557004047611, CONTENT: There’s never been a better time to create exceptional search experiences. By leveraging the capabilities of LLMs and generative AI, we can predict user intent, improve relevance, surface timely content, 

In [8]:
cur.close()
conn.close()